In [197]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import copy

df_hospital=pd.read_csv(r'../data/hospital_resources/Hospital_resources_hanmin.csv', encoding='ISO-8859-1')
df_hospital.head()

# fig = go.Figure()
# fig.add_trace(go.Scattergeo(
#     locationmode = 'USA-states',
#     lon = df_hospital['Longitude'],
#     lat = df_hospital['Latitude'],
#     hoverinfo = 'text',
#     text = df_hospital['Inst'],
#     mode = 'markers',
#     marker = dict(
#         size = 5,
#         color = 'black',
# #         line = dict(
# #             width = 3,
# #             color = 'rgba(68, 68, 68, 0)'
# #         )
#     )
#     ))

# fig.update_layout(
# #     directed=False
#     title_text = 'Resource reallocation',
#     showlegend = False,
#     geo = dict(
#         resolution = 50, 
#         scope = 'europe',
# #         center = {'lat': (data.lat.min() + data.lat.max())/2,
# #                 'lon': (data.lon.min() + data.lon.max())/2},
#         projection_type = 'azimuthal equal area',
#         showland = True,
#         landcolor = 'rgb(243, 243, 243)',
#         countrycolor = 'rgb(204, 204, 204)',
#     ),
# )

,ID,KT,Inst,Adr,PersA,PersP,BettenStatA,pBettenBelStatA,ICU_beds,Ventilators,Latitude,Longitude,Unnamed: 12
0,0,BS,UniversitÃ_tsspital Basel,Hebelstrasse 32,913.50,1595.22,633.000000,92.463806,38,28,47.561196,7.581541,NaN
1,1,GE,Les HÃÂ«pitaux Universitaires de GenÃÂ¬ve HUG,Rue GabrielleÃ¥_PerretÃ¥_Gentil 4,1829.32,4498.96,1054.282192,85.289218,63,47,46.193711,6.148856,NaN
2,2,VD,CHUV Centre Hospitalier Universitaire Vaudois,Rue du Bugnon 21,1653.73,3400.37,929.000000,82.588437,56,41,46.524749,6.639659,NaN
3,3,ZH,UniversitÃ_tsspital ZÃ_rich,RÃ_mistrasse 100,1221.75,2574.21,953.884931,78.680694,57,42,47.376736,8.549199,NaN
4,4,BE,Insel Gruppe AG (universitÃ_r),Freiburgstrasse 18,1355.66,2739.25,876.298630,83.329008,53,39,46.947455,7.424801,NaN


In [ ]:
ven_aloc_solu = np.load(r'../data/hospital_resources/Transported_ventilators.npy', allow_pickle=True)
up_tri = np.triu(ven_aloc_solu, 1)
lw_tri = np.tril(ven_aloc_solu, -1)
full_dec_mat = lw_tri-up_tri.transpose()  # use to decide who is sender who is receiver
sender_index = np.nonzero(ven_aloc_solu.sum(axis=1)>0)
receiver_index = np.nonzero(ven_aloc_solu.sum(axis=0)>0)
neutral_index = np.where(full_dec_mat.sum(axis=1) == 0)

sen_hosp = df_hospital.copy()
rec_hosp = df_hospital.copy()
neu_hosp = df_hospital.copy()

sen_hosp = sen_hosp.iloc[sender_index]
rec_hosp = rec_hosp.iloc[receiver_index]
neu_hosp = neu_hosp.iloc[neutral_index]

print(neu_hosp)

In [ ]:
print(ven_aloc_solu)

In [210]:
fig = go.Figure()
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df_hospital['Longitude'],
    lat = df_hospital['Latitude'],
    hoverinfo = 'text',
    text = df_hospital['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'black',
#         line = dict(
#             width = 3,
#             color = 'rgba(68, 68, 68, 0)'
#         )
    )
    ))
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = sen_hosp['Longitude'],
    lat = sen_hosp['Latitude'],
    hoverinfo = 'text',
    text = sen_hosp['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'green',
#         line = dict(
#             width = 3,
#             color = 'rgba(68, 68, 68, 0)'
#         )
    )
    ))

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = rec_hosp['Longitude'],
    lat = rec_hosp['Latitude'],
    hoverinfo = 'text',
    text = rec_hosp['Inst'],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'red',
#         line = dict(
#             width = 3,
#             color = 'rgba(68, 68, 68, 0)'
#         )
    )
    ))
# print(sen_hosp)
# print(rec_hosp)
# print(neu_hosp)
# receiver_hosp
# neutral_hosp = df_hospital[neutral_index[:]]

In [204]:
df_hospital

,ID,KT,Inst,Adr,PersA,PersP,BettenStatA,pBettenBelStatA,ICU_beds,Ventilators,Latitude,Longitude,Unnamed: 12
0,0,BS,UniversitÃ_tsspital Basel,Hebelstrasse 32,913.50,1595.22,633.000000,92.463806,38,28,47.561196,7.581541,NaN
1,1,GE,Les HÃÂ«pitaux Universitaires de GenÃÂ¬ve HUG,Rue GabrielleÃ¥_PerretÃ¥_Gentil 4,1829.32,4498.96,1054.282192,85.289218,63,47,46.193711,6.148856,NaN
2,2,VD,CHUV Centre Hospitalier Universitaire Vaudois,Rue du Bugnon 21,1653.73,3400.37,929.000000,82.588437,56,41,46.524749,6.639659,NaN
3,3,ZH,UniversitÃ_tsspital ZÃ_rich,RÃ_mistrasse 100,1221.75,2574.21,953.884931,78.680694,57,42,47.376736,8.549199,NaN
4,4,BE,Insel Gruppe AG (universitÃ_r),Freiburgstrasse 18,1355.66,2739.25,876.298630,83.329008,53,39,46.947455,7.424801,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,230,GE,Clinique de Carouge,Avenue CardinalÃ¥_Mermillod 1,14.78,40.46,15.000000,28.566210,1,1,46.186931,6.142615,NaN
63,257,ZH,UniversitÃ_ts-Kinderspital ZÃ_rich das Spita...,Steinwiesstrasse 75,302.29,683.98,167.128767,75.649979,10,7,47.372031,8.557767,NaN
64,258,SG,Ostschweizer Kinderspital,Claudiusstrasse 6,85.25,272.69,74.000000,81.728989,4,3,47.431105,9.387304,NaN
65,259,BS,UniversitÃ_ts-Kinderspital beider Basel (UKBB),Spitalstrasse 33,157.55,411.82,118.501370,72.873558,7,5,47.563624,7.581830,NaN


In [214]:
non_zero_ind = np.nonzero(ven_aloc_solu)
sen_ind = non_zero_ind[0]
rec_ind = non_zero_ind[1]

data = {'orig_lat': [-1.1]*len(rec_ind), 
        'orig_lon': [-1.1]*len(rec_ind), 
        'dest_lat': [-1.1]*len(rec_ind),
        'dest_lon': [-1.1]*len(rec_ind),
        'orig_hosp': ['']*len(rec_ind),
        'dest_hosp': ['']*len(rec_ind),
        'num_ven_tansp': [0]*len(rec_ind)} 
df_ven_reloc = pd.DataFrame(data)
ind = 0
for orig_ind, dest_ind in zip(sen_ind,rec_ind):
        print(df_hospital['Latitude'][orig_ind])
        df_ven_reloc['orig_lat'][ind] = df_hospital['Latitude'][orig_ind]
        df_ven_reloc['orig_lon'][ind] = df_hospital['Longitude'][orig_ind]
        df_ven_reloc['orig_hosp'][ind] = df_hospital['Inst'][orig_ind]  # from hospital
        
        df_ven_reloc['dest_lat'][ind] = df_hospital['Latitude'][dest_ind]
        df_ven_reloc['dest_lon'][ind] = df_hospital['Longitude'][dest_ind]
        df_ven_reloc['dest_hosp'][ind] = df_hospital['Inst'][dest_ind]  # to hospital
        df_ven_reloc['num_ven_tansp'][ind] = ven_aloc_solu[orig_ind, dest_ind]
        ind += 1

print(df_ven_reloc)


47.561196
47.561196
46.524749
46.524749
47.376736
47.376736
46.947455
46.947455
47.389185
47.454285999999996
47.391652
46.440108
46.440108
46.77032
46.77032
46.522437
46.522437
47.711995
47.711995
47.263268
47.263268
47.200103999999996
47.200103999999996
47.322205
46.801389
46.801389
47.488122
47.488122
47.058078
47.058078
47.05812
47.05812
47.549937
47.549937
47.351971999999996
46.86411
46.86411
47.397628000000005
47.397628000000005
46.958811
46.958811
46.947645
46.95279
47.147814000000004
47.049215000000004
47.049215000000004
46.680833
47.399167
47.507967
47.34917
47.566617
47.566617
47.394227
47.366863
46.761733
47.43119
47.43119
47.194648
47.194648
47.194648
47.216521
47.216521
46.228776
46.383439
46.383439
47.189221
47.189221
47.372031
     orig_lat  orig_lon   dest_lat  dest_lon  \
0   47.561196  7.581541  47.389185  8.059995   
1   47.561196  7.581541  46.197616  9.026881   
2   46.524749  6.639659  46.193711  6.148856   
3   46.524749  6.639659  46.235100  7.387328   
4   47.37

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\PycharmProjects\ch_vs_virus_416\venv\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\caha\Py

In [215]:
df_ven_reloc.to_csv('../data/hospital_resources/df_ven_reloc.csv')

In [ ]:
# # loop df_hospital to create line every
# ind = 0
# for orig_ind in range(67):
#     for dest_ind in range(67):
#         df_ven_reloc['orig_lat', ind] = df_hospital['Latitude'][orig_ind].copy()
#         df_ven_reloc['orig_lon', ind] = df_hospital['Longitude'][orig_ind].copy()
#         df_ven_reloc['orig_hosp', ind] = df_hospital['Inst'][orig_ind]  # from hospital
        
#         df_ven_reloc['dest_lat', ind] = df_hospital['Latitude'][dest_ind].copy()
#         df_ven_reloc['dest_lon', ind] = df_hospital['Longitude'][dest_ind].copy()  
#         df_ven_reloc['dest_hosp', ind] = df_hospital['Inst'][dest_ind]  # to hospital
#         df_ven_reloc['num_ven_tansp', ind] = ven_aloc_solu[orig_ind, dest_ind]
#         ind += 1
#         print(ind)
# # print(df_ven_reloc)    

In [216]:
for i in range(len(df_ven_reloc)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df_ven_reloc['orig_lon'][i], df_ven_reloc['dest_lon'][i]],
            lat = [df_ven_reloc['orig_lat'][i], df_ven_reloc['dest_lat'][i]],
            mode = 'lines',
            line = dict(width = 1.5,color = 'blue'),
            opacity = float(df_ven_reloc['num_ven_tansp'][i]) / float(df_ven_reloc['num_ven_tansp'].max()),
        )
    )

fig.update_layout(
    title_text = 'Ventilator reallocation results',
    showlegend = False,
    geo = dict(
        resolution = 50, 
        scope = 'europe',
#         center = {'lat': (df_ven_reloc['dest_lat'].min() + df_ven_reloc['dest_lat'].max())/2,
#                   'lon': (df_ven_reloc['orig_lon'].min() + df_ven_reloc['orig_lon'].max())/2},
        projection_type = 'stereographic',
#         showland = True,
        showlakes = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()